In [1]:
import os
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
import json

In [2]:
# get date today in mm-dd-yyyy format
today = datetime.today().strftime('%m-%d-%Y')
today

'05-30-2020'

In [3]:
# get all stocks
stocks_url = 'https://www.pse.com.ph/stockMarket/companyInfoSecurityProfile.html?method=getListedRecords&common=yes&ajax=true'
stocks_request = requests.get(stocks_url, headers={'Referer':'https://www.pse.com.ph/stockMarket/listedCompanyDirectory.html'}).json()
content = json.dumps(stocks_request, indent = 4, sort_keys=True)
json_content = json.loads(content)
stock_data = json_content['records']
stocks = []
for item in stock_data:
    # i can alread run data here
    stocks.append([
        item['companyId'],
        item['companyName'],
        item['indexName'],
        item['listingDate'],
        item['securityName'],
        item['securitySymbol'],
        item['securitySymbolId'],
        item['securityType'],
        item['subsectorName']
    ])

stocks_df = pd.DataFrame(stocks)
stocks_df.columns = ['companyId','companyName','indexName','listingDate','securityName','securitySymbol','securitySymbolId','securityType','subsectorName']

stocks_df.rename(columns={
    'companyId': 'company_id',
    'companyName': 'company_name',
    'indexName': 'index_name',
    'listingDate': 'listing_date',
    'securityName': 'security_name',
    'securitySymbol': 'security_symbol',
    'securitySymbolId': 'security_symbol_id',
    'securityType': 'security_type',
    'subsectorName': 'subsector_name'
},inplace=True)
stocks_df['listing_date'] = stocks_df['listing_date'].apply(pd.to_datetime)
stocks_df.index = stocks_df['security_symbol_id']
stocks_df.drop('index_name', axis=1, inplace=True)
stocks_df.to_csv('stocks.csv',index=False)

In [10]:
# define function to retrieve stock data
url = 'https://edge.pse.com.ph/common/DisclosureCht.ax'
def get_stock_data(companyName=None, companyId=None, securityId=None, endDate=None, startDate=None):
    # define referer in the headers
    referer = 'https://edge.pse.com.ph/companyPage/stockData.do?cmpy_id=%s' % securityId
    # define data to be sent to server
    json = { 'cmpy_id': companyId, 'security_id': securityId, 'endDate': endDate, 'startDate': startDate }
    # define headers
    headers = { 'Content-Type':'application/json', 'Referer': referer }
    # call request
    security_data = requests.post(url, headers=headers, json=json).json()
    # return json data
    chart_data = security_data['chartData']
    chart_data_df = pd.DataFrame(chart_data)
    chart_data_df.columns = ['open','value','close','chart_date','high','low']
    chart_data_df = chart_data_df[['chart_date','open','high','low','close','value']]
    filename = 'stocks/%s.csv' % companyName
    chart_data_df.to_csv(filename,index=False)

In [14]:
for stock in stocks:
    try:
        get_stock_data(stock[5],stock[0],stock[6],today,'01-01-2009')
        print('done', stock[5])
        time.sleep(5)
    except ValueError:
        print('Something went wrong', stock[5])

done 2GO
done 8990P
done HOUSE
done BRN
done ANS
done ABS
done ABSP
done ACEPH
done ACEX
done APC
done ATN
done ATNB
done ABA
done AEV
done AP
done AR
done ACE
done ANI
done HOME
done AGI
done FOOD
done ACR
done ALHI
done APO
done APX
done APL
done ARA
done ALCO
done ALCPB
done ALCPC
done AAA
done AUB
done ABG
done ATI
done AT
done AB
done AXLM
done AC
done ACPA
done ACPB1
done APB2R
done ALI
done ALLHC
done BLFI
done BDO
done BH
done BPI
done BSC
done BEL
done BC
done BCB
done BCP
done BCOR
done BLOOM
done BMM
done BHI
done BKR
done COL
done CEB
done CHI
done CLI
done CPV
done CPVB
done CHP
done CAT
done CEU
done CNPF
done CPM
done CPG
done CPGP
done C
done CIP
done CHIB
done TECB2
done TECH
done LAND
done CDC
done CSB
done COAL
done CIC
done CA
done CAB
done COSCO
done CROWN
done CEI
done CYBR
done DNL
done DMW
done DFNN
done DITO
done DMC
Something went wrong DMCP
done DAVIN
done DELM
done DMPA1
done DMPA2
done DWC
done DIZ
done DD
done DDPR
done EEI
done EAGLE
done EW
done ECP
done